In [1]:
import glob
import pandas as pd
import numpy as np
import re
import spacy
import gensim

from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter
from nltk.tokenize import TreebankWordTokenizer

kTOKENIZER = TreebankWordTokenizer()
nlp = spacy.load('en')

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
training_set = pd.read_csv('./data/ROCStories__spring2016 - ROCStories_spring2016.csv')

In [3]:
training_set.head()

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,9a51198e-96f1-42c3-b09d-a3e1e067d803,Overweight Kid,Dan's parents were overweight.,Dan was overweight as well.,The doctors told his parents it was unhealthy.,His parents understood and decided to make a c...,They got themselves and Dan on a diet.
1,617e7ada-3878-488d-bd56-40695b91f053,The Bike Accident,Carrie had just learned how to ride a bike.,She didn't have a bike of her own.,Carrie would sneak rides on her sister's bike.,She got nervous on a hill and crashed into a w...,The bike frame bent and Carrie got a deep gash...
2,79b0da1f-e460-4173-ba58-8c9e2553c53a,Beach,Morgan enjoyed long walks on the beach.,She and her boyfriend decided to go for a long...,"After walking for over a mile, something happe...",Morgan decided to propose to her boyfriend.,Her boyfriend was upset he didn't propose to h...
3,d173b7de-4611-4cdf-934c-912834755e41,The bad customer.,Jane was working at a diner.,"Suddenly, a customer barged up to the counter.",He began yelling about how long his food was t...,Jane didn't know how to react.,"Luckily, her coworker intervened and calmed th..."
4,af0fd5a4-de36-47ba-8aa2-e99d10986d7a,Being Patient,I was talking to my crush today.,She continued to complain about guys flirting ...,I decided to agree with what she says and list...,"After I got home, I got a text from her.",She asked if we can hang out tomorrow.


In [12]:
training_pos = []
training_neg = []

for i in range(len(training_set)):
    sentence = ''
    for j in range(4):
        s = 'sentence' + str(j + 1)
        c = training_set.iloc[i][s]
        sentence += c
    training_pos.append((gensim.utils.simple_preprocess(sentence) , gensim.utils.simple_preprocess(training_set.loc[i]['sentence5'])))
    
    r = np.random.randint(0,len(training_set))
    while r == i:
        r = np.random.randint(0,len(training_set))
    training_pos.append((gensim.utils.simple_preprocess(sentence) , gensim.utils.simple_preprocess(training_set.loc[r]['sentence5'])))


In [13]:
training_pos_4only = []
training_neg_4only = []

for i in range(len(training_set)):
    sentence=training_set.iloc[i]['sentence4']
    
    training_pos_4only.append((gensim.utils.simple_preprocess(sentence) , gensim.utils.simple_preprocess(training_set.loc[i]['sentence5'])))
    
    r = np.random.randint(0,len(training_set))
    while r == i:
        r = np.random.randint(0,len(training_set))
    training_neg_4only.append((gensim.utils.simple_preprocess(sentence) , gensim.utils.simple_preprocess(training_set.loc[r]['sentence5'])))

In [16]:
# training_pos[:10]
# training_pos_4only[:10]

In [6]:
test_data = pd.read_csv('./data/cloze_test_test__spring2016 - cloze_test_ALL_test.csv')

In [7]:
test_data.head()

,InputStoryid,InputSentence1,InputSentence2,InputSentence3,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2,AnswerRightEnding
0,b929f263-1dcd-4a0b-b267-5d5ff2fe65bb,My friends all love to go to the club to dance.,They think it's a lot of fun and always invite.,I finally decided to tag along last Saturday.,I danced terribly and broke a friend's toe.,My friends decided to keep inviting me out as ...,"The next weekend, I was asked to please stay h...",2
1,7cbbc0af-bcce-4f56-871d-963f9bb6a99d,I tried going to the park the other day.,The weather seemed nice enough for a walk.,Within minutes of getting there I started snee...,My eyes were watery and it was hard to breathe.,My allergies were too bad and I had to go back...,It reminded me of how much I loved spring flow...,1
2,4745d627-be9b-45f2-ad92-99c82cc83f85,Avery was married with children.,She was tired of her boring life.,"One day, she decided to meet up with an old bo...",She made poor decisions that night and was unf...,Avery thought her children would be happy with...,Avery regretted what she did the next day.,2
3,6bc5a855-3a02-454c-8d8b-4428b1864f95,Josh loved when his mom baked apple pie.,He hated how he always had to wait until after...,So he decided this time he would sneak a piece...,The eggs his mom used must have been bad though.,Josh thought that the pie was delicious.,Josh got sick.,2
4,2f74e81d-957b-4541-9e66-59e5eaa6aef1,John was writing lyrics for his new album.,He started experiencing writer's block.,He tried to force himself to write but it woul...,"He took a walk, hung out with some friends, an...",He felt inspiration and then went back home to...,John then got an idea for his painting.,1


In [10]:
test_1 = []
test_2 = [] 

for i in range(len(test_data)):
    sentence = ''
    for j in range(4):
        s = 'InputSentence'+str(j+1)
        c = test_data.iloc[i][s]
        sentence += c

    test_1.append((gensim.utils.simple_preprocess(sentence) ,gensim.utils.simple_preprocess(test_data.loc[i]["RandomFifthSentenceQuiz1"])))
    test_2.append((gensim.utils.simple_preprocess(sentence) ,gensim.utils.simple_preprocess(test_data.loc[i]["RandomFifthSentenceQuiz2"]))) 

In [11]:
test_1_4only = []
test_2_4only = [] 

for i in range(len(test_data)):
    test_1_4only.append((gensim.utils.simple_preprocess(test_data.loc[i]['InputSentence4']) , gensim.utils.simple_preprocess(test_data.loc[i]["RandomFifthSentenceQuiz1"])))
    test_2_4only.append((gensim.utils.simple_preprocess(test_data.loc[i]['InputSentence4']) , gensim.utils.simple_preprocess(test_data.loc[i]["RandomFifthSentenceQuiz2"])))

In [17]:
list_pos=[(doc, 1) for doc in training_pos_4only]
list_neg=[(doc, -1) for doc in training_neg_4only]
list_pos_t=[(doc, 1) for doc in test_1_4only]
list_neg_t=[(doc, -1) for doc in test_2_4only]



# list_pos=[(doc, 1) for doc in training_pos]
# list_neg=[(doc, -1) for doc in training_neg]

# list_pos_t=[(doc, 1) for doc in test_1]
# list_neg_t=[(doc, -1) for doc in test_2]



train_list=list_pos+list_neg
train_targets, train_regressors = zip(*train_list)
test_list=list_pos_t+list_neg_t
test_targets, test_regressors = zip(*test_list)

array([1, 2, 3, 4, 5])

In [18]:
from gensim.models import Doc2Vec

In [66]:
model = Doc2Vec.load('./word2vec_model/m1.d2v')

In [67]:
X = []
for i in range(len(train_targets)):
    X.append(np.append(model.infer_vector(train_targets[i][0]),model.infer_vector(train_targets[i][1])))
train_x = np.asarray(X)

In [68]:
Y = np.asarray(train_regressors)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y)
train_y = le.transform(Y)
np.mean(train_y)

0.5

In [74]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(max_iter=1000)
logreg.fit(train_x, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [83]:
test_list = []
for i in range(len(test_targets)):
    test_list.append(np.append(model.infer_vector(test_targets[i][0]),model.infer_vector(test_targets[i][1])))
test_x = np.asarray(test_list)
# test_Y = np.asarray(test_regressors)
# test_y = le.transform(test_Y)

In [84]:
preds=[]
probs = logreg.predict_proba(test_x)
half=int(len(probs)/2)
for i in range(half):
    if probs[i][1]>=probs[i+half][1]:
        preds.append(1)
    else:
        preds.append(2)

In [85]:
test_answer_true=np.array(list(test_data["AnswerRightEnding"]))

In [86]:
np.mean(np.array(preds)==test_answer_true)

0.5227151256012827

2400 model, all, 0.5269909139497595 <br>
2400 model, 4, 0.5173703901656868

In [ ]:
test_x